In [4]:
#%run EP3_Notebook.ipynb
import numpy as np
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt

In [12]:
### definição da geometria ###

# Pontos notáveis ([x, y])
A = np.array([ 0,  0])
B = np.array([ 5,  0])
C = np.array([ 9,  0])
D = np.array([18,  0])
E = np.array([36,  0])
F = np.array([36, 18])
G = np.array([36, -5]) # ponto criado artificialmente
H = np.array([45,  0]) # ponto criado artificialmente
I = np.array([65,  0]) # ponto criado artificialmente

dx = 1

# Dimensões vigas/treliças
A1 = ( 1.8, 0.9) # torre
A2 = ( 0.9, 0.9) # ponte
D3 = (0.05,)     # treliças

def discretiza(ponto1, ponto2, dx, dx_list=[1, 2, 3]):
    """
    Função que discretiza um elemento, dado dois pontos a ele pertencentes.
    Além disso, respeita a imposição de que se não houver como gerar o elemento
    usa um dx menor, e apenas um, da lista de opções.
    """
    #lista de opções
    dx_list = np.array(dx_list)
    dx_menores = dx_list[dx_list <= dx]
    
    #definição de onde é a discretização
    if ponto1[1] == ponto2[1]:
        #tem coordenadas y iguais
        #a discretização é em x
        pto1 = ponto1[0]
        pto2 = ponto2[0]
        orientacao = 'x'
    elif ponto1[0] == ponto2[0]:
        #tem coordenadas x iguais
        #a discretização é em y
        pto1 = ponto1[1]
        pto2 = ponto2[1]
        orientacao = 'y'
    else:
        #tem coordenadas x e y distintas
        print("discretização não horizontal/vertical")
        return 0
    
    #discretizando em dx até o valor que der
    discretizacao = np.arange(pto1, pto2, dx)
    
    #pegando o quanto falta para a discretização
    diff = pto2 - discretizacao[-1]
    
    if diff != 0:
        #se a discretização não foi perfeita
        if diff in dx_menores:
            #se tem um valor dx que encaixe no espaço que 'sobra'
            discretizacao = np.append(discretizacao, discretizacao[-1] + diff)
        else:
            # há um ponto que não encaixa e não temos um dx adequado para ele
            print("Erro na discretização!")
        
    #colocando o vetor com as duas coordenadas
    if orientacao == 'x':
        discretizacao = np.array([np.array([x, ponto1[1]]) for x in discretizacao])
    elif orientacao == 'y':
        discretizacao = np.array([np.array([ponto1[0], y]) for y in discretizacao])
        
    return discretizacao
    

def discretiza_elemento(point_list, dx):
    elemento = []
    for i, ponto in enumerate(point_list[:-1]):
        discretizacao = discretiza(ponto, point_list[i+1], dx)
        if i+2 != len(point_list):
            # se não for o último elemento
            elemento.extend(discretizacao[:-1])
            # não se coloca o último ponto para evitar repetição
        else:
            # se for o último elemento
            elemento.extend(discretizacao)
    return elemento


# Plataforma (viga)
# A-B-C-D-E-H-I
plataforma_points = [A, B, C, D, E, H, I]
plataforma = discretiza_elemento(plataforma_points, dx)

# Torre (viga)
# G-E-F
torre_points = [G, E, F]
torre = discretiza_elemento(torre_points, dx)

# Cabos (treliça)
# 1: B-F
cabo1_points = [B, F]
# 2: D-F
cabo2_points = [D, F]
# 3: I-F
cabo3_points = [I, F]


In [1]:
#old funcs
def plot_struct3(dict_elementos):
    """
    Função que plota para a visulização da estrutura montada.
    Ela plota todos os nós assim como os elementos que estão ligando-os.
    
    
    #depois vou deixar bonita a docstring
    dict_elementos: dicionário no qual as chaves são as cores (strings)
    e os elementos são as listas de nós.
    """
    for color in dict_elementos.keys():
        for elementos in dict_elementos[color]:
            #coloca-se os nós
            x, y = elementos
            plt.scatter(x, y, facecolors='none', s=4, edgecolors=color)
            #coloca-se a linha ligando os pontos
            for i in range(0, len(x)):
                plt.plot(x[i:i+2], y[i:i+2], color+'-')

def plot_struct2(dict_elementos):
    """
    Função que plota para a visulização da estrutura montada.
    Ela plota todos os nós assim como os elementos que estão ligando-os.

    """
    for estrutura in dict_elementos.keys():
        dados  = dict_elementos[estrutura]
        color  = dados['color']
        points = dados['points']
        x, y = zip(*points)
        #plota-se os nós
        plt.scatter(x, y, facecolors='none', s=4, edgecolors=color)
        #coloca-se a linha ligando os pontos
        for i in range(0, len(x)):
                plt.plot(x[i:i+2], y[i:i+2], color+'-')